In [1]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np

In [90]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
    '_session_id2': 'e2801ab93fb91a13e46c00233a052a93',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2; _session_id2=e2801ab93fb91a13e46c00233a052a93',
    'If-None-Match': 'W/"d0bdc5d451dc0582cacda7aca354c860"',
    'Referer': 'https://www.goodreads.com/review/list/127864674-rachael?utf8=%E2%9C%93&order=d&sort=review&view=reviews&title=rachael&per_page=100',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [91]:
def clean_title(raw_title):
    if raw_title is None:
        return 0
    result = re.search(r"(.*)\n(\W*)(.*)", raw_title)
    if result is None:
        return 0
    else:
        return result.group(3)

def clean_isbn(raw_isbn):
    if raw_isbn is None:
        return 0
    result = re.search(r"(isbn13) *(\d*)\n", raw_isbn)
    if result == None:
        return 0
    else:
        return result.group(2)

def clean_rating(raw_rating):
    if raw_rating is None:
        return 0
    result = re.search(r"(.*)\n(\W*)(.*)", raw_rating)
    if result is None:
        return 0
    else:
        return result.group(3)

In [141]:
books = []
for page in range(1,8):
    response = requests.get(f"https://www.goodreads.com/review/list/52491901-lena?order=d&page={page}&per_page=100&sort=review&utf8=%E2%9C%93&view=reviews", cookies=cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('tr', class_ = "bookalike review")
    id = 6
    print(len(book_container))
    for book in book_container:
        title = clean_title(book.find('td', class_ = "field title").text)
        isbn = clean_isbn(book.find('td', class_ = "field isbn13").text)
        rating = clean_rating(book.find('td', class_ = "field rating").text)
        book_data = [id, title, isbn, rating]
        books.append(book_data)
books = pd.DataFrame(books, columns=["UserID", "Title", "ISBN", "Rating"])
books.head()


100
100
100
100
100
100
100


,UserID,Title,ISBN,Rating
0,6,Six of Crows,9781627795098,it was amazing
1,6,Empire of Storms,9781408872895,it was amazing
2,6,A Court of Wings and Ruin,,it was amazing
3,6,To Kill a Kingdom,9781250112682,really liked it
4,6,A Reaper at the Gates,9780448494500,it was amazing


In [142]:
books.to_csv("UserRanking_Lena.csv", index = False)

In [145]:
df = pd.read_csv("joinedUserReview.csv")

clean = df[df['Rating'].notna()]
clean.loc[clean['Rating']== 'did not like it', 'Rating'] = 1
clean.loc[clean['Rating']== 'it was ok', 'Rating'] = 2
clean.loc[clean['Rating']== 'liked it', 'Rating'] = 3
clean.loc[clean['Rating']== 'really liked it', 'Rating'] = 4
clean.loc[clean['Rating']== 'it was amazing', 'Rating'] = 5



In [146]:
clean.to_csv("joinedUserReview.csv", index=False)

In [ ]:
data = pd.read_csv('joinedUserReview.csv')
